In [20]:
import ete3
from ete3 import Tree
import os

def reroot_tree(tree_name):
    tree_name_only = tree_name.split(".")[0]
    base_dir = "delta_statistics/rerooted_trees/" + tree_name_only + "/"
    leaf_dir = base_dir + "leaves/"
    inner_dir = base_dir + "inner_nodes/"
    
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
        
    if not os.path.exists(inner_dir):
        os.makedirs(inner_dir)
        
    if not os.path.exists(leaf_dir):
        os.makedirs(leaf_dir)
    
    t = Tree("data/trees/" + tree_name)
    t.resolve_polytomy()
    # to get all possible rooting on terminal branches
    ls = []
    for leaf in t:
        ls.append(leaf.name)
    for i in ls:
        t.set_outgroup(t&i)
        new_tree_path = leaf_dir + tree_name_only + "_root=" + i + ".tree"
        with open(new_tree_path, "w+") as tree_file:
            tree_file.write(t.write()+"\n")
    t = Tree("data/trees/" + tree_name)
    t.resolve_polytomy()
    # to get all possible rooting on internal branches
    edge = 0
    ancestor = []
    
    for node in t.traverse():
        if not node.is_leaf() and not node.is_root():
            node.name = "NODE_%d" %edge
            ancestor.append(node.name)
            edge += 1
    for i in ancestor:
        t.set_outgroup( t&i )
        new_tree_path = inner_dir + tree_name_only + "_root=" + i + ".tree"
        with open(new_tree_path, "w+") as tree_file:
            tree_file.write(t.write()+"\n")
    #root in original position        
    t = Tree("data/trees/" + tree_name)
    t.resolve_polytomy()
    i = "NODE_%d" %edge
    new_tree_path = inner_dir + tree_name_only + "_root=" + i + ".tree"
    with open(new_tree_path, "w+") as tree_file:
        tree_file.write(t.write()+"\n")

reroot_tree("geo_duration.tree")